# DAG with "Bad" Control: Model 12 from *A Crash Course in Good and Bad Controls*
## 210521

![](Model12.png)

In [1]:
from causalsage import *

In [2]:
%display latex

In [3]:
for i in range(4):
    for j in range(4):
        var('w%d%d' % (i, j))

# Compute Overall Covariance Matrix from DAG

$$W_{ij} = W_{i \gets j}$$

In [13]:
W = matrix([[0, 0, 0, 0], # to 0
            [w10, 0, 0, 0], # to 1
            [0, w21, 0, 0], # to 2
            [0, w31, 0, 0]]) # to 3
    
W

[  0   0   0   0]
[w10   0   0   0]
[  0 w21   0   0]
[  0 w31   0   0]

In [14]:
Cov_total = Sigma_from_W(W)

Cov_total

[                      1                     w10                 w10*w21                 w10*w31]
[                    w10               w10^2 + 1         w10^2*w21 + w21         w10^2*w31 + w31]
[                w10*w21         w10^2*w21 + w21 w10^2*w21^2 + w21^2 + 1 w10^2*w21*w31 + w21*w31]
[                w10*w31         w10^2*w31 + w31 w10^2*w21*w31 + w21*w31 w10^2*w31^2 + w31^2 + 1]

In [15]:
[Cov_total[i, i] for i in range(4)]

[1, w10^2 + 1, w10^2*w21^2 + w21^2 + 1, w10^2*w31^2 + w31^2 + 1]

# Compute Partial Covariance Matrix from DAG

In [22]:
## Example: Compute Cov(0, 2)

targets = [0, 2]
condset = []

partial_covariance_matrix(targets, condset, Cov_total)

[                      1                 w10*w21]
[                w10*w21 w10^2*w21^2 + w21^2 + 1]

## Example: Compute Cov(0, 2 | 3)

In [21]:
targets = [0, 2]
condset = [3]

partial_covariance_matrix(targets, condset, Cov_total)

[                                     -w10^2*w31^2/(w10^2*w31^2 + w31^2 + 1) + 1           w10*w21 - (w10^2*w21*w31 + w21*w31)*w10*w31/(w10^2*w31^2 + w31^2 + 1)]
[          w10*w21 - (w10^2*w21*w31 + w21*w31)*w10*w31/(w10^2*w31^2 + w31^2 + 1) w10^2*w21^2 + w21^2 - (w10^2*w21*w31 + w21*w31)^2/(w10^2*w31^2 + w31^2 + 1) + 1]

# Compute Partial Regression Coefficients from DAG

$$\beta_{YX \cdot Z} = \frac{\gamma_{YX \cdot Z}}{\sigma_{X \cdot Z}^{2}}$$

## $\beta_{20}$ vs. $\beta_{20 \cdot 3}$

Here, conditioning on $X_{3}$ biases the coefficient away from the causal effect of $X_{0}$ on $X_{2}$.

In [18]:
partial_regression_coefficient(response=2, predictor=0, condset=[], Sigma=Cov_total).full_simplify()

w10*w21

In [19]:
partial_regression_coefficient(response=2, predictor=0, condset=[3], Sigma=Cov_total).full_simplify()

w10*w21/(w31^2 + 1)